In [83]:
import pandas as pd
import numpy as np
import random

In [98]:
class Simulator():
    """
    Class to generate fake data
    TODO: figure out seed setting
    """
    
    def __init__(self, num_rows: int, seed = 42):
        """
        Initialize an empty Pandas DataFrame
        """
        
        self.num_rows = num_rows
        self.seed = np.random.seed(seed)
    
    # def get_num_rows(self):
    #     return self.num_rows
    
    def generate_categorical_column(self, num_categories: int):
        """
        Returns a Pandas Series with num_rows
        """
        return pd.Series(data = self.generate_categorical_distribution(num_categories), index = range(self.num_rows), dtype= 'object')
    
    def generate_categorical_distribution(self, num_categories: int, name: str = 'color', distribution = 'uniform'):
        """
        Specify a categorical feature's distribution
        By default the distribution is uniform - all categories will have an equal chance of being drawn
        User can also pass a size [num_categories] array of probabilities for each category
        """

        categories = [name + "_" + str(i+1) for i in range(4)]
        if distribution == 'uniform':
            np.random.seed(self.seed)
            categorical_column = np.random.choice(categories, size = self.num_rows)
        
        return categorical_column





In [99]:
a = Simulator(num_rows = 100)

In [102]:
a.generate_categorical_column(num_categories = 4)

0     color_3
1     color_1
2     color_2
3     color_4
4     color_2
       ...   
95    color_2
96    color_4
97    color_2
98    color_4
99    color_3
Length: 100, dtype: object